In [31]:
import torch
import cv2
import pandas as pd
from tqdm import tqdm
from PIL import Image, ImageDraw
from matplotlib import pyplot as plt
import pybboxes as pbx

In [60]:
# load model
model = torch.hub.load("./yolov7", 
                       'custom', 
                       "./data/best.pt", 
                       source='local', 
                       force_reload=True)
model.eval()


                 from  n    params  module                                  arguments                     
  0                -1  1       928  models.common.Conv                      [3, 32, 3, 1]                 
  1                -1  1     18560  models.common.Conv                      [32, 64, 3, 2]                
  2                -1  1     36992  models.common.Conv                      [64, 64, 3, 1]                
  3                -1  1     73984  models.common.Conv                      [64, 128, 3, 2]               
  4                -1  1      8320  models.common.Conv                      [128, 64, 1, 1]               
  5                -2  1      8320  models.common.Conv                      [128, 64, 1, 1]               
  6                -1  1     36992  models.common.Conv                      [64, 64, 3, 1]                
  7                -1  1     36992  models.common.Conv                      [64, 64, 3, 1]                
  8                -1  1     36992  

Adding autoShape... 


autoShape(
  (model): Model(
    (model): Sequential(
      (0): Conv(
        (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU()
      )
      (1): Conv(
        (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(64, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU()
      )
      (2): Conv(
        (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(64, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU()
      )
      (3): Conv(
        (conv): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(128, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU()
      )
 

In [33]:
# test infernece

img_path = 'data/test_images_phase1/0a4a4ea7-a9bf-41d9-a027-e8e4bc3a483d.jpeg'

img = cv2.imread(img_path)
print(img.shape)
# print(img)
img = cv2.cvtColor(img, cv2.COLOR_BGRA2RGB)

results = model(img, size=704)
result_pd = results.pandas().xyxy[0].iloc[0]
print(result_pd)
# for i in range(len(result_pd)):
xmin, ymin = int(result_pd['xmin']), int(result_pd['ymin'])
xmax, ymax = int(result_pd['xmax']), int(result_pd['ymax'])
confidence = result_pd['confidence']
print(confidence)
name = result_pd['name']
print(name)
img = cv2.rectangle(img, (xmin, ymin), (xmax, ymax), (36,255,12), 1)
cv2.putText(img, f"{name} : {confidence:0.2f}", (xmin, ymin-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (36,255,12), 2)
    
plt.imshow(img)
plt.show()
# img = img.save("test.jpg")
plt.savefig('test.png')

(1966, 1347, 3)
xmin            464.25766
ymin           564.821106
xmax           853.404297
ymax          1147.549805
confidence       0.882047
class                   3
name                culex
Name: 0, dtype: object
0.882047176361084
culex


C:\Users\vedan\AppData\Local\Temp\ipykernel_17356\3057900828.py:24: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()


In [40]:
df = pd.read_csv("data/test_phase1_v2.csv")

In [46]:
df['img_fName'].nunique()

2676

In [67]:
TEST_IMG_PATH = "data/test_images_phase1/"
output_file = []
result_list = []
for i,row in tqdm(df.iterrows()):
    img_path = TEST_IMG_PATH+row['img_fName']
    # print(img_path)
    img = cv2.imread(img_path)
    # print(img)
    img = cv2.cvtColor(img, cv2.COLOR_BGRA2RGB)
    results = model(img, size=704)
    result_pd = results.pandas().xyxy[0].head(1)#.iloc[0]
    if len(result_pd) == 0:
        output_file.append([row['img_fName'], row['img_w'], row['img_h'], 0, 0, 0, 0, 'albopictus'])
    else:
    # result_list.append(img)
    # for i in range(len(result_pd)):
    #     xmin, ymin = int(result_pd['xmin'][i]), int(result_pd['ymin'][i])
    #     xmax, ymax = int(result_pd['xmax'][i]), int(result_pd['ymax'][i])
    #     name = result_pd['name'][i]
    #     output_file.append([row['img_fName'], row['img_w'], row['img_h'], xmin, ymin, xmax, ymax, name])
        # for i in range(len(result_pd)):
        xmin, ymin = int(result_pd['xmin'][0]), int(result_pd['ymin'][0])
        xmax, ymax = int(result_pd['xmax'][0]), int(result_pd['ymax'][0])
        name = result_pd['name'][0]
        output_file.append([row['img_fName'], row['img_w'], row['img_h'], xmin, ymin, xmax, ymax, name])

output_df = pd.DataFrame(output_file, columns=['img_fName', 'img_w', 'img_h', 'bbx_xtl', 'bbx_ytl', 'bbx_xbr', 'bbx_ybr', 'class_label'])
output_df.to_csv("test_set_phase_1.csv", index=False)
    

# yolo_bbox1 = (0.286972, 0.647157, 0.404930, 0.371237)
# yolo_bbox2 = (0.681338, 0.366221, 0.454225, 0.418060)
# W, H = 300, 300  # WxH of the image
# pbx.convert_bbox(yolo_bbox1, from_type="yolo", to_type="voc", image_size=(W, H))
# >>> (25, 138, 147, 250)
# pbx.convert_bbox(yolo_bbox2, from_type="yolo", to_type="voc", image_size=(W, H))
# >>> (136, 47, 273, 173)

#     name = result_pd['name']
#     print(name)
#     img = cv2.rectangle(img, (xmin, ymin), (xmax, ymax), (36,255,12), 1)
#     cv2.putText(img, f"{name} : {confidence:0.2f}", (xmin, ymin-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (36,255,12), 2)
    



2676it [40:04,  1.11it/s]


In [61]:
results = model(result_list[-1], size=704)

In [66]:
results.pandas().xyxy#[0]#.head(1)
plt.imshow(result_list[-1])
plt.show()
# img = img.save("test.jpg")
plt.savefig('test.png')

C:\Users\vedan\AppData\Local\Temp\ipykernel_17356\2751841299.py:3: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()


In [49]:
output_df1 = output_df.drop_duplicates()

In [51]:
output_df1 = df.merge(output_df, on='img_fName', how='inner')

In [73]:
TEST_IMG_PATH = "data/test_images_phase1/"
output_file = []
result_list = []
for i,row in tqdm(df.iterrows()):
    img_path = TEST_IMG_PATH+row['img_fName']
    # print(img_path)
    img = cv2.imread(img_path)
    # print(img)
    img = cv2.cvtColor(img, cv2.COLOR_BGRA2RGB)
    results = model(img, size=704)
    result_pd = results.pandas().xyxy[0]#.head(1)#.iloc[0]
    if len(result_pd) == 0:
        output_file.append([row['img_fName'], row['img_w'], row['img_h'], 0, 0, 0, 0, 'albopictus'])
    elif len(result_pd) >1:
        outs = result_pd.iloc[result_pd['confidence'].idxmax()]
        xmin, ymin = int(outs['xmin']), int(outs['ymin'])
        xmax, ymax = int(outs['xmax']), int(outs['ymax'])
        name = outs['name']
        output_file.append([row['img_fName'], row['img_w'], row['img_h'], xmin, ymin, xmax, ymax, name])
    else:
    # result_list.append(img)
    # for i in range(len(result_pd)):
    #     xmin, ymin = int(result_pd['xmin'][i]), int(result_pd['ymin'][i])
    #     xmax, ymax = int(result_pd['xmax'][i]), int(result_pd['ymax'][i])
    #     name = result_pd['name'][i]
    #     output_file.append([row['img_fName'], row['img_w'], row['img_h'], xmin, ymin, xmax, ymax, name])
        # for i in range(len(result_pd)):
        xmin, ymin = int(result_pd['xmin'][0]), int(result_pd['ymin'][0])
        xmax, ymax = int(result_pd['xmax'][0]), int(result_pd['ymax'][0])
        name = result_pd['name'][0]
        output_file.append([row['img_fName'], row['img_w'], row['img_h'], xmin, ymin, xmax, ymax, name])

output_df = pd.DataFrame(output_file, columns=['img_fName', 'img_w', 'img_h', 'bbx_xtl', 'bbx_ytl', 'bbx_xbr', 'bbx_ybr', 'class_label'])
output_df.to_csv("test_set_phase_1_3.csv", index=False)
    

# yolo_bbox1 = (0.286972, 0.647157, 0.404930, 0.371237)
# yolo_bbox2 = (0.681338, 0.366221, 0.454225, 0.418060)
# W, H = 300, 300  # WxH of the image
# pbx.convert_bbox(yolo_bbox1, from_type="yolo", to_type="voc", image_size=(W, H))
# >>> (25, 138, 147, 250)
# pbx.convert_bbox(yolo_bbox2, from_type="yolo", to_type="voc", image_size=(W, H))
# >>> (136, 47, 273, 173)

#     name = result_pd['name']
#     print(name)
#     img = cv2.rectangle(img, (xmin, ymin), (xmax, ymax), (36,255,12), 1)
#     cv2.putText(img, f"{name} : {confidence:0.2f}", (xmin, ymin-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (36,255,12), 2)
    



995it [12:33,  1.52it/s]

In [69]:
output_df['img_fName'][1]

'f207780c-0d0a-4d41-bd95-92ca53ff81a3.jpeg'